In [12]:
import time

from hashlib import sha256
import json

class Block:
  
    def __init__(self, index, transactions, timestamp, previous_hash, nonce=0):
    # Todo for students, implement the initialization of the properties for this class.
        self.index = index
        self.transactions = transactions
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.nonce = nonce
        #print("good")
    def compute_hash(self):
        block_string = json.dumps(self.__dict__, sort_keys=True)
         #todo for students: implement a sha256 solution to return the hex digest of the block_string, and return it. 
        return sha256(block_string.encode()).hexdigest()

class Blockchain(Block):

    def __init__(self):
        # student todo: write the code for unconfirmed_transactions property.
        self.unconfirmed_transactions = []
        # student todo: write the code for chain property.
        self.chain = []
        # student todo: add and initialize the difficulty property. 
        self.difficulty = 2
        self.create_genesis_block()
 
    def create_genesis_block(self):
        genesis_block = Block(0, [], time.time(), "0")
        genesis_block.hash = genesis_block.compute_hash()
        self.chain.append(genesis_block)
       


    # todo define get_last_block(self) -> Block, hint: consider an index to the last element in your chain property.
    def last_block(self):
        return self.chain[-1]
        
    
    def proof_of_work(self, block):
        # computed_hash = how do we get the hash from a Block object?
        computed_hash = block.compute_hash()
        while not computed_hash.startswith('0' * self.difficulty):
            block.nonce += 1
            computed_hash = block.compute_hash()
        # Don't forget to return the hash. 
        return computed_hash

    # todo define is_valid_proof(self, block:Block, block_hash:str) -> bool
    def is_valid_proof(self, block, block_hash):
        # hint, we want to return true if block_hash.startswith('0' * Blockchain.difficulty)
        # AND if block_hash is in fact the hash of our block (maybe use block.compute_hash())
        return (block_hash.startswith('0' * self.difficulty) and
                block_hash == block.compute_hash())
    
    # todo define add_block(self, block:Block, proof:str) -> bool
    # hints, we probably want to get the previous hash from get_last_block()
    def add_block(self, block, proof):
        previous_hash = self.chain[-1]
        previous_hash= previous_hash.hash
        # Check if the previous hash matches the previous hash in the block argument. 
        if previous_hash != block.previous_hash:
            return False
        # Make use of is_valid_proof for block and proof to check if someone is trying to do something sneaky.
        #return False early and return control the calling function if the block is not valid.
        if not self.is_valid_proof(block, proof):
            return False
        block.hash = proof
        self.chain.append(block)#So let's add the hash to our block, append it to the chain,and return True.
        return True

    # todo, define add_new_transaction(self, transaction:dict) -> None
    # Hint how do we append to a list? How do we do this for a property of our object (self). Ease one line function. 
    def add_new_transaction(self, transaction):
        self.unconfirmed_transactions.append(transaction)# append to a list
     
      
    def mine(self):
        if not self.unconfirmed_transactions:
            return False
        # consider retrieving the last block to a local variable make life easy. 
        last_block = self.chain[-1]
        self.difficulty = 2
        # Let's make sure we use our Block constructor to create a new block with all the transactions we want to mine 
        new_block = Block(index=last_block.index + 1,
                          transactions=self.unconfirmed_transactions,
                          timestamp=time.time(),
                          previous_hash=last_block.hash)
        # Finally, let's be sure to use our handy proof_of_work function, add_block function, and to remember to reset our
        # unconfirmed_transactions (our mem-pool), before returning our new block index;
        proof = self.proof_of_work(new_block)
        self.add_block(new_block, proof)
        self.unconfirmed_transactions = []
        return new_block.index
    


In [14]:
blockchain=Blockchain()
#b0=blockchain.create_genesis_block()
t1=blockchain.add_new_transaction(['Satoshi','Mike','5 BTC'])
t2=blockchain.add_new_transaction(['Mike','Satoshi','1 BTC'])
t3=blockchain.add_new_transaction(['Satoshi','Hal Finney','5 BTC'])
blockchain.mine()

1

In [11]:
t4=blockchain.add_new_transaction(['Mona','Eyad','10 BTC'])
t5=blockchain.add_new_transaction(['Eyad','Satoshi','1 BTC'])
t6=blockchain.add_new_transaction(['Satoshi','Eyad','20 BTC'])
blockchain.mine()

2

In [16]:
t7=blockchain.add_new_transaction(['Mona','Eyad','10 BTC'])
t8=blockchain.add_new_transaction(['shehanh','Satoshi','1 BTC'])
t9=blockchain.add_new_transaction(['Satoshi','Eyad','20 BTC'])
blockchain.mine()

3